In [1]:
!pip install keras_vggface

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [46]:
import os
import numpy as np
import tensorflow as tf
import keras
from keras.engine import  Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras import optimizers
from keras_vggface.vggface import VGGFace
from keras.callbacks import ReduceLROnPlateau

In [4]:
print(tf.__version__)
print(keras.__version__)

1.15.2
2.3.1


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Preparation

In [6]:
!ls "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/"

fer2013.csv  PrivateTest.zip  PublicTest.zip  Training.zip


In [7]:
!ls 

drive  sample_data  Test  Training  Validation


In [8]:
# This creates a temporary folder in drive root folder, so it will have to be reloaded again when required after terminating the session
# Permanent data are stored in dataset in zipped format
# This copies the zipped file and store in root of google drive temporarily
! rm -rf Training; mkdir Training
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/Training.zip" -d Training

! rm -rf Validation; mkdir Validation
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/PublicTest.zip" -d Validation

! rm -rf Test; mkdir Test
! unzip -q "drive/My Drive/ENGR635-Deep Learning System Design Project/Dataset/fer2013/PrivateTest.zip" -d Test

In [9]:
!ls

drive  sample_data  Test  Training  Validation


In [10]:
!ls -l Training/

total 808
drwxr-xr-x 2 root root 122880 Oct 11 05:04 Angry
drwxr-xr-x 2 root root  12288 Oct 11 05:04 Disgust
drwxr-xr-x 2 root root 131072 Oct 11 05:04 Fear
drwxr-xr-x 2 root root 200704 Oct 11 05:04 Happy
drwxr-xr-x 2 root root 135168 Oct 11 05:04 Neutral
drwxr-xr-x 2 root root 135168 Oct 11 05:04 Sad
drwxr-xr-x 2 root root  90112 Oct 11 05:04 Surprise


In [11]:
%%bash
root='Training/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

Angry: 3995
Disgust: 436
Fear: 4097
Happy: 7215
Neutral: 4965
Sad: 4830
Surprise: 3171


In [12]:
%%bash
root='Validation/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

Angry: 467
Disgust: 56
Fear: 496
Happy: 895
Neutral: 607
Sad: 653
Surprise: 415


In [13]:
%%bash
root='Test/'
IFS=$(echo -en "\n\b")
(for dir in $(ls -1 "$root")
    do printf "$dir: " && ls -i "$root$dir" | wc -l
 done)

Angry: 491
Disgust: 55
Fear: 528
Happy: 879
Neutral: 626
Sad: 594
Surprise: 416


In [14]:
train_dir = "Training/"
validation_dir = "Validation/"
test_dir = "Test/"

## All data are ready

In [71]:
# https://github.com/rcmalli/keras-vggface#projects--blog-posts
conv_base = VGGFace(model='vgg16',
                  include_top = False,
                  input_shape = (100,100,3))

In [72]:
conv_base.summary()

Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 100, 100, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 100, 100, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 50, 50, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 50, 50, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 50, 50, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 25, 25, 128)     

In [73]:
DROP_OUT_RATE = 0.5
FROZEN_LAYER_NUM = len(conv_base.layers)
FROZEN_LAYER_NUM

19

In [74]:
print("Number of trainable weights before freezing the conv base:", len(conv_base.trainable_weights))

Number of trainable weights before freezing the conv base: 26


In [75]:
# conv_base.trainable = False
for i in range(FROZEN_LAYER_NUM):
    conv_base.layers[i].trainable = False

In [76]:
print("Number of trainable weights after freezing the conv base:", len(conv_base.trainable_weights))

Number of trainable weights after freezing the conv base: 0


In [77]:
print(conv_base.get_layer('conv5_3').trainable)

False


In [78]:
conv_base.summary()

Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 100, 100, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 100, 100, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 50, 50, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 50, 50, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 50, 50, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 25, 25, 128)     

In [79]:
last_layer = conv_base.get_layer('pool5').output

x = Flatten(name='flatten')(last_layer)
x = Dropout(DROP_OUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc6')(x)
# x = Dropout(DROP_OUT_RATE)(x)
# x = Dense(256, activation='relu', name='fc7')(x)
out = Dense(7, activation='softmax', name='classifier')(x)

model = Model(conv_base.input, out)

# optim = keras.optimizers.Adam(lr=ADAM_LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# #optim = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
# sgd = keras.optimizers.SGD(lr=SGD_LEARNING_RATE, momentum=0.9, decay=SGD_DECAY, nesterov=True)
# rlrop = keras.callbacks.ReduceLROnPlateau(monitor='val_acc',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001)
model.compile(optimizer=optimizers.Adam(learning_rate=0.001,), loss='categorical_crossentropy', metrics=['accuracy'])

In [80]:
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 100, 100, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 100, 100, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 50, 50, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 50, 50, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 50, 50, 128)       147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 25, 25, 128)       0  

In [81]:
# Image Data Generator setup
train_datagen = ImageDataGenerator(rescale=1./255,
                                   featurewise_center=False,
                                   featurewise_std_normalization=False,
                                   rotation_range=90,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True
                                   )
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (100, 100),
    batch_size = 50,
    shuffle = True,
    class_mode='categorical'
)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size = (100, 100),
    batch_size = 50,
    class_mode = 'categorical'
)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=28709//50,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=3589//50,
    shuffle=True,
    use_multiprocessing=True,
    callbacks=[reduce_lr]
)

Epoch 1/30
574/574 [==============================] - 83s 144ms/step - loss: 1.6109 - accuracy: 0.3645 - val_loss: 1.1812 - val_accuracy: 0.4715
Epoch 2/30
574/574 [==============================] - 82s 143ms/step - loss: 1.5437 - accuracy: 0.3984 - val_loss: 1.3531 - val_accuracy: 0.4936
Epoch 3/30
574/574 [==============================] - 83s 144ms/step - loss: 1.5115 - accuracy: 0.4115 - val_loss: 1.2803 - val_accuracy: 0.5092
Epoch 4/30
574/574 [==============================] - 83s 145ms/step - loss: 1.4907 - accuracy: 0.4214 - val_loss: 1.5067 - val_accuracy: 0.5211
Epoch 5/30
566/574 [============================>.] - ETA: 1s - loss: 1.4725 - accuracy: 0.4304

In [44]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
loss = history.history['loss']

val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'r', label="Training accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation accuracy")
plt.title('Training and Validation Accuracy with freezed Conv base of VGG16 trained on VGGFACE2')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'r', label="Training accuracy")
plt.plot(epochs, val_loss, 'b', label="Validation accuracy")
plt.title('Training and Validation Loss with freeze conv base of VGG16 trained on VGGFACE2')
plt.legend()
plt.show()

NameError: ignored